# 预训练模型

**causal language model** pretrain case

## step1 import lib

In [1]:
from datasets import load_dataset, Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, DataCollatorForLanguageModeling, TrainingArguments, Trainer, BloomForCausalLM

## step2 load datasets

In [2]:
ds = Dataset.load_from_disk("./wiki_cn_filtered/")

In [3]:
ds

Dataset({
    features: ['source', 'completion'],
    num_rows: 10000
})

In [4]:
ds[0]

{'source': 'wikipedia.zh2307',
 'completion': "西安交通大学博物馆（Xi'an Jiaotong University Museum）是一座位于西安交通大学的博物馆，馆长是锺明善。\n历史\n2004年9月20日开始筹建，2013年4月8日正式建成开馆，位于西安交通大学兴庆校区陕西省西安市咸宁西路28号。建筑面积6,800平米，展厅面积4,500平米，馆藏文物4,900余件。包括历代艺术文物馆、碑石书法馆、西部农民画馆、邢良坤陶瓷艺术馆、陕西秦腔博物馆和书画展厅共五馆一厅。\n营业时间\n* 周一至周六：上午九点至十二点，下午一点至五点\n* 周日闭馆"}

## step3 data process

In [5]:
model_path = r"D:\CodeLibrary\huggingface_model\Langboat\bloom-389m-zh"
tokenizer = AutoTokenizer.from_pretrained(model_path)

def process_func(examples):
    contents = [e + tokenizer.eos_token for e in examples["completion"]]    # 加上 [EOS] 结束符
    return tokenizer(contents, max_length=384, truncation=True)

In [6]:
tokenized_ds = ds.map(process_func, batched=True, remove_columns=ds.column_names)
tokenized_ds

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 10000
})

In [7]:
from torch.utils.data import DataLoader

dl = DataLoader(tokenized_ds, batch_size=2, collate_fn=DataCollatorForLanguageModeling(tokenizer, mlm=False))   # causal model， mlm=False

In [8]:
next(enumerate(dl))

(0,
 {'input_ids': tensor([[    3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
              3,     3

In [9]:
tokenizer.pad_token, tokenizer.pad_token_id

('<pad>', 3)

In [10]:
tokenizer.eos_token, tokenizer.eos_token_id

('</s>', 2)

## step4 create model

In [11]:
model = AutoModelForCausalLM.from_pretrained(model_path)

e:\software\anaconda3\envs\transformers\Lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


## step5 config train parameters

In [12]:
args = TrainingArguments(
    output_dir="./causal_lm",
    per_device_train_batch_size=2,
    gradient_accumulation_steps=16,
    logging_steps=10,
    num_train_epochs=1,
    fp16=True
)

## step6 create trainer

In [13]:
trainer = Trainer(
    args=args,
    model=model,
    tokenizer=tokenizer,
    train_dataset=tokenized_ds,
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False)
)

## step7 model train

In [14]:
trainer.train()

Step,Training Loss
10,4.055300
20,3.963200
30,3.931900
40,3.835700
50,3.857600
60,3.752700
70,3.679600
80,3.707300
90,3.666300
100,3.638200


TrainOutput(global_step=312, training_loss=3.5985299394680905, metrics={'train_runtime': 11076.6577, 'train_samples_per_second': 0.903, 'train_steps_per_second': 0.028, 'total_flos': 6092709626314752.0, 'train_loss': 3.5985299394680905, 'epoch': 0.9984})

## step8 model inference

In [15]:
from transformers import pipeline

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer, device=0)

In [16]:
# do_sample参数
pipe("西安交通大学博物馆（Xi'an Jiaotong University Museum）是一座位于西安", max_length=128, do_sample=True)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


[{'generated_text': "西安交通大学博物馆（Xi'an Jiaotong University Museum）是一座位于西安交通大学大唐芙蓉池遗址遗址公园内的西安交通大学博物馆，由长安古建博物馆和中国西安城墙博物馆相结合的开放博物馆。\n历史\n根据西安城市规划局的西安城市规划，西安交通大学大唐芙蓉池遗址遗址公园内修建了西安交通大学博物馆。博物馆位于大唐芙蓉池遗址内，建筑面积近4万平米，为西安市内规模最大的大型博物馆。西安城规建设时，将西安交通大学博物馆作为文物建筑纳入西安市历史文化名城，被定位为西安市历史文化名城核心区之一"}]

In [17]:
pipe("下面是一则游戏新闻。小编报道，近日，游戏产业发展的非常", max_length=128, do_sample=True)

[{'generated_text': '下面是一则游戏新闻。小编报道，近日，游戏产业发展的非常顺利，近日，游戏相关产业发展，而且从2016年开始，中国游戏研发已经非常成功。\n由于游戏产业比较发达、经济效益可观，故而越来越多的人加入到了游戏的浪潮中去。而对于这游戏产业来讲，并不是只有中国游戏公司可以参与该游戏产业的竞争，比如欧美的一些大型游戏公司也有参与中国游戏公司研发的游戏。但是由于这些游戏公司开发了比较好的题材，所以大部分的中小企业并没有参与中国游戏产业的竞争。并且由于现在的中小企业在游戏开发和营销方面比'}]